## Prerequisites

To execute this tutorial you will need:

* pyiceberg
* pyiceberg[glue]
* pandas
* pyarrow
* datetime

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [2]:
from pyiceberg.catalog import load_catalog

## 1. Manipulate the Glue Data Catalog

Print all databases in the Glue Data Catalog

In [3]:
catalog = load_catalog("glue", **{"type": "glue"})

print(catalog.list_namespaces())

[('iceberg',)]


Print all tables in the Glue Data Catalog iceberg database

In [4]:
tables = catalog.list_tables('iceberg')

print(tables)

[('iceberg', 'sampledataicebergtable')]


Print the metadata for the sampledata_iceberg_gdc table

In [5]:
table = catalog.load_table("iceberg.sampledataicebergtable")

print(f"Location: {table.metadata.location}")
print(f"Last Updated: {table.metadata.last_updated_ms}")
print(f"Current Snapshot Id: {table.metadata.current_snapshot_id}")
print(f"Schema: {table.metadata.schemas}")
print("...")

# Optional print full metadata
# print(table.metadata)

Location: s3://pyiceberg-gdc-s3-chclwg0acebi/iceberg/iceberg.db/sampledataicebergtable
Last Updated: 1765032152056
Current Snapshot Id: 6085099039381742923
Schema: [Schema(NestedField(field_id=1, name='id', field_type=StringType(), required=False), NestedField(field_id=2, name='name', field_type=StringType(), required=False), NestedField(field_id=3, name='create_date', field_type=StringType(), required=False), NestedField(field_id=4, name='last_update_time', field_type=TimestamptzType(), required=False), schema_id=0, identifier_field_ids=[])]
...


## 2. Read an Iceberg Table

In [6]:
table = catalog.load_table("iceberg.sampledataicebergtable")

scan = table.scan()

Write output to pyarrow

In [7]:
arrow_scan = scan.to_arrow()

print(arrow_scan)

pyarrow.Table
id: string
name: string
create_date: string
last_update_time: timestamp[us, tz=UTC]
----
id: [["6","2","1","3"],["4"]]
name: [["Adam","Will","Christopher","Emmeline"],["John"]]
create_date: [["2020-01-01","2020-01-01","2020-01-01","2020-01-01"],["2020-01-01"]]
last_update_time: [[2020-01-01 00:00:00.000000Z,2020-01-01 00:00:00.000000Z,2020-01-02 00:00:00.000000Z,2020-01-02 00:00:00.000000Z],[2020-01-01 00:00:00.000000Z]]


Write output to pandas

In [8]:
pandas_scan = scan.to_pandas()

print(pandas_scan)

  id         name create_date          last_update_time
0  6         Adam  2020-01-01 2020-01-01 00:00:00+00:00
1  2         Will  2020-01-01 2020-01-01 00:00:00+00:00
2  1  Christopher  2020-01-01 2020-01-02 00:00:00+00:00
3  3     Emmeline  2020-01-01 2020-01-02 00:00:00+00:00
4  4         John  2020-01-01 2020-01-01 00:00:00+00:00


## 3. Append to an Iceberg Table

In [9]:
from datetime import datetime, timezone
import pyarrow

In [10]:
append_df = pyarrow.Table.from_pylist(
    [
        {"id": "7", "name": "Jim", "create_date": "2020-01-01", "last_update_time": datetime.now(timezone.utc)},
        {"id": "8", "name": "Anna", "create_date": "2020-01-01", "last_update_time": datetime.now(timezone.utc)} 
    ],
)

append = table.append(append_df)

Check if the append worked

In [11]:
scan = table.scan()

pandas_scan = scan.to_pandas()

print(pandas_scan.sort_values("id"))

  id         name create_date                 last_update_time
4  1  Christopher  2020-01-01        2020-01-02 00:00:00+00:00
3  2         Will  2020-01-01        2020-01-01 00:00:00+00:00
5  3     Emmeline  2020-01-01        2020-01-02 00:00:00+00:00
6  4         John  2020-01-01        2020-01-01 00:00:00+00:00
2  6         Adam  2020-01-01        2020-01-01 00:00:00+00:00
0  7          Jim  2020-01-01 2025-12-06 14:44:00.097111+00:00
1  8         Anna  2020-01-01 2025-12-06 14:44:00.097118+00:00


## 4. Upsert to an Iceberg Table

In [12]:
upsert_df = pyarrow.Table.from_pylist(
    [
        {"id": "7", "name": "Will-Update", "create_date": "2020-01-01", "last_update_time": datetime.now(timezone.utc)},
        {"id": "9", "name": "Sara", "create_date": "2020-01-02", "last_update_time": datetime.now(timezone.utc)} 
    ],
)

upsert = table.upsert(
    upsert_df,
    join_cols=["id"],                 
    when_matched_update_all=True,     # update existing rows
    when_not_matched_insert_all=True  # insert new rows
)

print("Upsert result:", upsert)

Upsert result: UpsertResult(rows_updated=1, rows_inserted=1)


Check if the upsert worked

In [13]:
scan = table.scan()

pandas_scan = scan.to_pandas()

print(pandas_scan.sort_values("id"))

  id         name create_date                 last_update_time
5  1  Christopher  2020-01-01        2020-01-02 00:00:00+00:00
4  2         Will  2020-01-01        2020-01-01 00:00:00+00:00
6  3     Emmeline  2020-01-01        2020-01-02 00:00:00+00:00
7  4         John  2020-01-01        2020-01-01 00:00:00+00:00
3  6         Adam  2020-01-01        2020-01-01 00:00:00+00:00
1  7  Will-Update  2020-01-01 2025-12-06 14:44:07.067683+00:00
2  8         Anna  2020-01-01 2025-12-06 14:44:00.097118+00:00
0  9         Sara  2020-01-02 2025-12-06 14:44:07.067687+00:00
